In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0679],
        [0.0464]], grad_fn=<AddmmBackward>)

In [2]:
print(net[2].state_dict()) #通过索引访问任意层

OrderedDict([('weight', tensor([[ 0.1430,  0.3284,  0.3050,  0.2955,  0.1587,  0.2102, -0.2544, -0.0309]])), ('bias', tensor([0.0612]))])


In [5]:
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

Parameter containing:
tensor([0.1416], requires_grad=True)
tensor([0.1416])
None


In [12]:
# 一次性访问所有参数 列表推导式
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

net.state_dict()['2.bias'].data

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([0.1416])

In [14]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套块进去，f字符串，主要是为了给每一层命名
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

print(rgnet)


Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
rgnet[0][1][0].bias.data

tensor([-9.3356e-02,  6.9690e-02,  1.6958e-04, -2.0502e-01,  4.5077e-01,
        -3.0777e-02,  7.3256e-02,  2.2617e-01])

## 自定义初始化

In [20]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        nn.init.zeros_(m.bias)

net[0].apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([42., 42., 42., 42.]), tensor(0.))

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            'Init',
            *[(name, params.shape) for name, params in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data += m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-4.1388, -4.6200,  1.4865, -3.5567],
        [ 9.6880,  1.5182,  0.7722,  7.4287],
        [ 1.8368,  1.0540,  7.1372,  8.6445],
        [ 6.8548, -1.4777, -4.8259,  0.1311],
        [10.4066,  1.2817, -5.1404,  2.2318],
        [ 1.5237,  2.1735, -4.5893,  2.2584],
        [-1.9241,  3.0487,  1.5577, -8.2835],
        [-5.0368,  6.5585, -4.4306,  8.2912]], requires_grad=True)

## 参数绑定（共享参数）

In [22]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0,0] = 150
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
